# CDSL ROS-Based Control Interface

Author: Sang-ik An (flyingwith@gmail.com)

This application has been tested only on the macOS M1.

## Linear System Simulator

### Introduction

In [ ]:
from IPython.display import Markdown, display
display(Markdown('../targets/linear_system_simulator/README.md'))

### Example 1

This example shows a basic usage of `ControlInterface`.

- The target is initialized when the `ControlInterface` class is instantiated.
  - It executes the shell script `simulation1_server`.
- The target is finalized when the `ControlInterface` class is deleted.
  - It executes the shell script `simulation1_close`.
- The target runs when the member function `run()` of `ControlInterface` is called.
  - It executes the shell script `simulation1_client`.
- A data directory is automatically created with the time stamp in the directory `${APP_DATA_DIR}`.
- The `parameter.xml` file is copied to the data directory and the result of `run()` is saved to the `data.csv` file.
- After `run()` returns, the data is accessible as member variables of the target.

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from interface import ControlInterface as CI

# check if ROS is running
if CI.is_ros_running():
    print('ROS is running')
else:
    print('ROS is not running')

# list all targets
CI.list_all_targets()

# print the parameter.xml file for the target
CI.print_xml('Linear System Simulator')

# initialize the target (simulation1_server runs)
ci = CI('Linear System Simulator')

# run the target (simulation1_client runs)
print('running simulation1...')
ci.run()
print('done')

# plot the result
fig, axs = plt.subplots(2,2)
axs[0,0].plot(ci.target.t, ci.target.x)
axs[0,0].set_title('state')
axs[0,1].plot(ci.target.t, ci.target.u)
axs[0,1].set_title('control input')
axs[1,0].plot(ci.target.t, ci.target.r, ci.target.t, ci.target.y)
axs[1,0].set_title('reference and output')
axs[1,1].plot(ci.target.t, ci.target.e)
axs[1,1].set_title('output tracking error')
fig.tight_layout()

# finalize the target (simulation1_close runs)
del ci

### Example 2

Various keywords arguments can be used when `ControlInteface` is instantiated or `run()` is called.

- `run=True`: When used in the instantiation of `ControlInterface`, the function `run()` is also called.
- `verbose=True`: When used in the instantiation of `ControlInterface`, the log messages are displayed.
- `data_dir_name=<name>`: When used in the instantiation of `ControlInterface` along with `run=True` or in the call of `run()`, the data directory name is set to `<name>`.

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt, os
from interface import ControlInterface as CI

ci = CI(0, run=True, data_dir_name='test', verbose=True)

print('\n'+'='*80)
print(f'data directory = {ci.target.data_dir}')
os.system(f'ls {ci.target.data_dir}')
print('='*80+'\n')

del ci

### Example 3

The parameters of the simulation can be set directly by modifying the parameter.xml or by the keywords arguments.

For array parameters, we can use various types:

- `np.ndarray`: `A = np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]])`
- `tuple` : `A = (0,1,0,0,0,1,0,0,0)`
- `list` : `A = [0,1,0,0,0,1,0,0,0]`
- `dict` : `A = {0:0,1:1,2:0,3:0,4:0,5:1,6:0,7:0,8:0}`
- `str` : `A = '0,1,0,0,0,1,0,0,0'`

If an array parameter has size 1, we can also use `int` or `float` type:

- `np.ndarray` : `Kp = np.array[[100]]`
- `int` : `Kp = 100`
- `float` : `Kp = 100.0`

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from interface import ControlInterface as CI

n = 3
m = 1
p = 1
A = np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]])
B = np.array([[0], [0], [1]])
C = np.array([[1, 0, 0]])
s0 = 0.0
r0 = np.array([1.0])
Kp = np.array([[100]])
Kd = np.array([[10]])
Ki = np.array([[1]])
ts = 0.001
t0 = 0.0
t1 = 3.0
x0 = np.array([0, 0, 0])

ci = CI(0, run=True, n=n, m=m, p=p, A=A, B=B, C=C, s0=s0, r0=r0, Kp=Kp, Kd=Kd, Ki=Ki, ts=ts, t0=t0, t1=t1, x0=x0)

fig, axs = plt.subplots(2,2)
axs[0,0].plot(ci.target.t, ci.target.x)
axs[0,0].set_title('state')
axs[0,1].plot(ci.target.t, ci.target.u)
axs[0,1].set_title('control input')
axs[1,0].plot(ci.target.t, ci.target.r, ci.target.t, ci.target.y)
axs[1,0].set_title('reference and output')
axs[1,1].plot(ci.target.t, ci.target.e)
axs[1,1].set_title('output tracking error')
fig.tight_layout()

del ci

### Example 4

We can run multiple simulations.

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from interface import ControlInterface as CI

fig, axs = plt.subplots(1)

ci = CI(0)

n = 10
for i in range(n):
    print(f'[{i}] running...')
    ci.run(Kp=50+100/n*i)
    if i == 0:
        axs.plot(ci.target.t, ci.target.r)
    axs.plot(ci.target.t, ci.target.y)

del ci

fig.tight_layout()

## Two-Link Manipulator

### Introduction

In [ ]:
from IPython.display import Markdown, display
display(Markdown('../targets/two_link_manipulator/README.md'))

### Example 1

Run the simulation without the modeling uncertainty.

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from interface import ControlInterface as CI

CI.print_xml("Two-Link Manipulator")

ci = CI('Two-Link Manipulator')

print('running simulation1...')
ci.run()
print('done')

fig, axs = plt.subplots(2,2)
axs[0,0].plot(ci.target.t, ci.target.r, ci.target.t, ci.target.q)
axs[0,0].set_title('joint position')
axs[0,1].plot(ci.target.t, ci.target.tau)
axs[0,1].set_title('joint torques')
axs[1,0].plot(ci.target.t, ci.target.dq)
axs[1,0].set_title('joint velocity')
axs[1,1].plot(ci.target.t, ci.target.e)
axs[1,1].set_title('tracking error')
fig.tight_layout()

del ci

### Example 2

Let's see the behavior of the controller according to the modeling uncertainty.

In [ ]:
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from interface import ControlInterface as CI

fig, axs = plt.subplots(1)

ci = CI(1)

n = 10
for i in range(n):
    print(f'[{i}] running...')
    ci.run(m1_e=9+2/n*i, m2_e=5.5-1/n*i)
    if i == 0:
        axs.plot(ci.target.t, ci.target.r)
    axs.plot(ci.target.t, ci.target.q)

del ci

fig.tight_layout()